## إعداد متغيرات البيئة
**ضع المتغيرات بدون \" \"**

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import clear_output
import os

# إنشاء حقول الإدخال
api_id_input = widgets.Text(
    placeholder='أدخل API_ID',
    description='API_ID:',
    value=''
)

api_hash_input = widgets.Text(
    placeholder='أدخل API_HASH',
    description='API_HASH:',
    value=''
)

session_input = widgets.Textarea(
    placeholder='أدخل SESSION',
    description='SESSION:',
    value='',
    layout=widgets.Layout(width='80%', height='100px')
)

bot_token_input = widgets.Text(
    placeholder='أدخل BOT_TOKEN',
    description='BOT_TOKEN:',
    value=''
)

# زر الحفظ
save_button = widgets.Button(
    description='حفظ المتغيرات',
    button_style='success',
    icon='save'
)

output = widgets.Output()

def save_variables(btn):
    with output:
        clear_output()
        
        # جمع القيم
        api_id = api_id_input.value.strip()
        api_hash = api_hash_input.value.strip()
        session = session_input.value.strip()
        bot_token = bot_token_input.value.strip()
        
        # التحقق من عدم وجود قيم فارغة
        if not all([api_id, api_hash, session, bot_token]):
            print("❌ يرجى ملء جميع الحقول!")
            return
        
        # كتابة المتغيرات في ملف .env داخل مجلد SiFThon
        env_content = f"""## ضع المتغيرات بدون \" \" ##
## ضع المتغيرات بدون \" \" ##

API_ID={api_id}
API_HASH={api_hash}
SESSION={session}
BOT_TOKEN={bot_token}
"""
        
        try:
            # إنشاء مجلد SiFThon إذا لم يكن موجوداً
            os.makedirs('SiFThon', exist_ok=True)
            
            # حفظ الملف داخل مجلد SiFThon
            env_path = os.path.join('SiFThon', '.env')
            with open(env_path, 'w', encoding='utf-8') as f:
                f.write(env_content)
            
            print("✅ تم حفظ المتغيرات بنجاح في ملف .env")
            print(f"📁 تم إنشاء الملف: {env_path}")
            print("📂 تم حفظ الملف داخل مجلد SiFThon")
        except Exception as e:
            print(f"❌ حدث خطأ أثناء الحفظ: {e}")

# ربط الدالة بالزر
save_button.on_click(save_variables)

# عرض العناصر
display(api_id_input)
display(api_hash_input)
display(session_input)
display(bot_token_input)
display(save_button)
display(output)

## تنزيل وتشغيل السكربت

In [ ]:
import requests
import zipfile
import os
import subprocess
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import clear_output

# إنشاء واجهة الإدخال
url_input = widgets.Text(
    placeholder='أدخل رابط ملف ZIP',
    description='رابط ZIP:',
    layout=widgets.Layout(width='80%')
)

download_button = widgets.Button(
    description='تحميل وتشغيل',
    button_style='info',
    icon='download'
)

process_output = widgets.Output()

def download_and_run(btn):
    with process_output:
        clear_output()
        
        url = url_input.value.strip()
        if not url:
            print("❌ يرجى إدخال رابط ملف ZIP")
            return
        
        try:
            print("⏳ جاري التحميل...")
            
            # تحميل الملف
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            # حفظ الملف المؤقت
            zip_filename = 'downloaded_file.zip'
            with open(zip_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            print("✅ تم التحميل بنجاح")
            print("⏳ جاري فك الضغط...")
            
            # فك الضغط
            with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
                zip_ref.extractall('.')
            
            print("✅ تم فك الضغط بنجاح")
            
            # التحقق من وجود مجلد SiFThon
            if not os.path.exists('SiFThon'):
                print("❌ لم يتم العثور على مجلد SiFThon بعد فك الضغط")
                return
            
            # التحقق من وجود ملف .env داخل SiFThon
            env_path = os.path.join('SiFThon', '.env')
            if not os.path.exists(env_path):
                print("⚠️  لم يتم العثور على ملف .env داخل مجلد SiFThon")
                print("يرجى ملء المتغيرات في الخلية الأولى أولاً")
                return
            
            print("📂 تم العثور على مجلد SiFThon وملف .env")
            print("⏳ جاري تثبيت المكتبات...")
            
            # تغيير المسار إلى مجلد SiFThon
            os.chdir('SiFThon')
            
            # تثبيت المكتبات
            if os.path.exists('requirements.txt'):
                result = subprocess.run(['pip', 'install', '-r', 'requirements.txt'], 
                                      capture_output=True, text=True)
                if result.returncode == 0:
                    print("✅ تم تثبيت المكتبات بنجاح")
                else:
                    print("❌ حدث خطأ في تثبيت المكتبات:")
                    print(result.stderr)
                    return
            else:
                print("⚠️  لم يتم العثور على ملف requirements.txt")
            
            print("🚀 جاري تشغيل السكربت...")
            
            # تشغيل السكربت
            result = subprocess.run(['python', '-m', 'SiFThon'], 
                                  capture_output=True, text=True)
            
            print("📋 مخرجات السكربت:")
            print(result.stdout)
            if result.stderr:
                print("❌ أخطاء:")
                print(result.stderr)
            
            # العودة إلى المسار الأصلي
            os.chdir('..')
            
            print("✅ اكتملت العملية")
            
        except requests.exceptions.RequestException as e:
            print(f"❌ خطأ في التحميل: {e}")
        except zipfile.BadZipFile:
            print("❌ الملف ليس ملف ZIP صالح")
        except Exception as e:
            print(f"❌ حدث خطأ غير متوقع: {e}")

# ربط الدالة بالزر
download_button.on_click(download_and_run)

# عرض العناصر
display(url_input)
display(download_button)
display(process_output)